In [23]:
import os
import CityHub
#import folium
import geopandas as gpd
import pandas as pd

import math
from statistics import mean

In [24]:
ch = CityHub.CityHub('São Paulo, São Paulo, Brazil')

Loading file...
Preprocessing city mesh...
Refining mesh...


In [26]:
G = ch.city_street_graph

In [27]:
ch.load_PALayer_mesh('data/sp/sp_setores_censitarios/35SEE250GC_SIR.shp','CD_GEOCODI',40.0)

Refining mesh...


0

In [28]:
#ch.load_PALayers_csv_data(0,'data/sp/census_data/sp_dados_demograficos_setores2010.csv','Cod_setor')
ch.load_PALayers_csv_data(0,'data/sp/census_data/data_preprocess.csv','Cod_setor')

True

In [29]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_pontoonibus/SAD69-96_SHP_pontoonibus.shp',23,'K',True,-45.0,-45.0)

0

In [30]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_estacaometro/SAD69-96_SHP_estacaometro_point.shp',23,'K')

1

In [31]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_estacaotrem/SAD69-96_SHP_estacaotrem.shp',23,'K')

2

In [32]:
ch.load_PBLayer('data/sp/infraestrutura/terminal_onibus/sad6996_terminal_onibus.shp',23,'K')

3

In [33]:
ch.load_PBLayer_csv_data('data/sp/crimes.csv','latitude','longitude',project_to_corners = True)

4

In [34]:
ch.save_preprocessed_CityHub('data/sp-4-abr-23.bin')

In [37]:
len(ch.city_vert_list)

121542

In [179]:
from shapely.geometry import Polygon, Point

city_vert_gdf = gpd.GeoDataFrame(list(range(0,len(ch.city_vert_list))),geometry = gpd.points_from_xy(pd.DataFrame(ch.city_vert_list)[0], pd.DataFrame(ch.city_vert_list)[1]))
city_vert_gdf.columns = ["vert", "geometry"]
city_vert_gdf

,vert,geometry
0,0,POINT (-23.56299 -46.70397)
1,1,POINT (-23.56746 -46.70223)
2,2,POINT (-23.57296 -46.69923)
3,3,POINT (-23.57183 -46.70207)
4,4,POINT (-23.53462 -46.74815)
...,...,...
121537,121537,POINT (-23.55463 -46.53176)
121538,121538,POINT (-23.57349 -46.49451)
121539,121539,POINT (-23.57281 -46.49286)
121540,121540,POINT (-23.56551 -46.48308)


In [180]:
#city_vert_gdf = city_vert_gdf.head(100)
#city_vert_gdf = city_vert_gdf.iloc[:2]
#city_vert_gdf

In [181]:
import types

def compute_layer_crimes(layer, ch, initial_date = None, final_date = None):

        tmp_layer_crimes = ch.PBLayers[layer].copy()
        if((not initial_date is None) or (not final_date is None)):
            tmp_layer_crimes["time"] = pd.to_datetime(tmp_layer_crimes["time"], format = "%Y/%m/%d", errors = 'coerce')
            tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes['time'].notna()]
            if(not initial_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["time"] >= initial_date]
            if(not final_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["time"] <= final_date]


        aux = pd.DataFrame(tmp_layer_crimes[["vert"]].value_counts().keys().tolist(), columns = ["vert"])

        aux["n"] = tmp_layer_crimes[["vert"]].value_counts().values
        #print(aux)
        #print(tmp_layer_crimes)
        
        df_vert = pd.DataFrame()
        df_vert['vert'] = list(ch.city_vert_dict.values())
        
        df_vert = df_vert.merge(aux, how='left', on='vert')
        df_vert['n'] = df_vert['n'].fillna(0)

        return df_vert

In [183]:
def compute_dynamic_feature_df(ch, initial_date = None, final_date = None):
    
    # crimes
    feature_df = compute_layer_crimes(layer = 4, ch = ch, initial_date = initial_date, final_date = final_date)
    
    return feature_df

In [187]:
import datetime

#Calcular X das features dinâmicas
x1 = datetime.datetime(2016, 5, 17)
x2 = datetime.datetime(2017, 5, 17)
X_dynamic = compute_dynamic_feature_df(ch,x1,x2)

In [188]:
X_dynamic

,vert,n
0,0,0.0
1,1,3.0
2,2,3.0
3,3,0.0
4,4,0.0
...,...,...
121537,121537,1.0
121538,121538,2.0
121539,121539,0.0
121540,121540,2.0


In [189]:
X_dynamic.n.sum()

109110.0

In [126]:
tmp_layer_crimes = ch.PBLayers[4].copy()

tmp_layer_crimes["time"] = pd.to_datetime(tmp_layer_crimes["time"], format = "%Y/%m/%d", errors = 'coerce')
tmp_layer_crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737856 entries, 0 to 737855
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  737856 non-null  int64         
 1   latitude    737856 non-null  object        
 2   longitude   737856 non-null  object        
 3   time        737856 non-null  datetime64[ns]
 4   latlong     737856 non-null  object        
 5   vert        737856 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 55.5+ MB


In [40]:
def compute_static_feature_vector(vert_ind, dist):

    feature_vec = pd.Series(dtype=float)

    layer_pontodeonibus = 0
    layer_estacaodemetro = 1
    layer_estacaodetrem = 2
    layer_terminaldeonibus = 3

    query_coords=ch.city_vert_list[vert_ind]

#       ibge
    ibge_list = pd.Series([0.0]*7)
    ibge_list.index = ['V001', 'V002', 'V003', 'V055', 'V056', 'V057', 'V065']

    for i in range(len(ibge_list.index)):
        feats = ch.polygon_features_from_PALayer_vertex(0, vert_ind,  ibge_list.index[i])
        if len(feats)==0:
            return pd.Series(dtype=float)
        for f in feats:
            if math.isnan(f):
                print('entrou onde eu quero')
                return pd.Series(dtype=float)
        ibge_list[i]=mean(feats)
    feature_vec = pd.concat([feature_vec, ibge_list])

#       transporte
    transporte_indices = ['Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus']
    transporte_list = pd.Series([0.0]*len(transporte_indices), index = transporte_indices)
    transporte_list['Pontos_de_onibus'] = len(ch.query_point_in_PBLayer(layer_pontodeonibus,query_coords[0],query_coords[1],dist))
    transporte_list['Estacao_de_metro'] = len(ch.query_point_in_PBLayer(layer_estacaodemetro,query_coords[0],query_coords[1],dist))
    transporte_list['Estacao_de_trem'] = len(ch.query_point_in_PBLayer(layer_estacaodetrem,query_coords[0],query_coords[1],dist))
    transporte_list['Terminal_de_onibus'] = len(ch.query_point_in_PBLayer(layer_terminaldeonibus,query_coords[0],query_coords[1],dist))

    feature_vec = pd.concat([feature_vec, transporte_list])

#       favelas
#    favela_indices = ['Favela_proxima']
#    favela_list = pd.Series([0.0]*len(favela_indices), index = favela_indices)
#    favela_list['Favela_proxima'] = (len(ch.query_point_in_RDLayer(0, query_coords[0],query_coords[1],0.5,True))>0)*1

#    feature_vec = pd.concat([feature_vec, favela_list])

    return feature_vec

In [41]:
X_static = city_vert_gdf["vert"].apply(lambda vert: compute_static_feature_vector(vert, 0.2))

In [42]:
X_static

,V001,V002,V003,V055,V056,V057,V065,Pontos_de_onibus,Estacao_de_metro,Estacao_de_trem,Terminal_de_onibus
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,265.0,265.333333,258.666667,17.666667,19.0,87.666667,22.666667,28.0,1.0,1.0,0.0
2,66.0,65.500000,46.000000,3.500000,1.5,9.500000,5.500000,3.0,0.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,149.0,149.000000,140.500000,11.500000,9.5,49.500000,2.500000,0.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,128.0,128.500000,125.500000,9.500000,6.5,39.000000,8.000000,0.0,0.0,0.0,0.0


In [ ]:
#list_vert_ind = list(range(0,len(ch.city_vert_list)))
#ch.city_vert_list
for i in range(0,20):
    teste = ch.polygon_features_from_PALayer_vertex(layer=0, vert_ind=i, feature='V001')
    print(teste)

In [ ]:
teste_get_dados()

In [ ]:
list_vert_ind = list(range(0,len(ch.city_vert_list)))

for vert_ind in list_vert_ind:
        feature_vec = pd.Series(dtype=float)
        query_coords=ch.city_vert_list[vert_ind]

        transporte_indices = ['Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus']
        transporte_list = pd.Series([0.0]*len(transporte_indices), index = transporte_indices)
        transporte_list['Pontos_de_onibus'] = len(ch.query_point_in_PBLayer(0,query_coords[0],query_coords[1],layer_pontodeonibus,0.2))
        transporte_list['Estacao_de_metro'] = len(ch.query_point_in_PBLayer(1,query_coords[0],query_coords[1],layer_estacaodemetro,0.2))
        transporte_list['Estacao_de_trem'] = len(ch.query_point_in_PBLayer(2,query_coords[0],query_coords[1],layer_estacaodetrem,0.2))
        transporte_list['Terminal_de_onibus'] = len(ch.query_point_in_PBLayer(3,query_coords[0],query_coords[1],layer_terminaldeonibus,0.2))

feature_vec = pd.concat([feature_vec, transporte_list])
feature_vec

In [127]:
G.number_of_nodes()

121542

In [128]:
G.number_of_edges()

177929

In [129]:
len(G)

121542

In [134]:
from networkx.algorithms import community

def get_communities(G):
    communities_generator = list(community.asyn_fluidc(G,4))
    communities_list = []
    for i in communities_generator:
        communities_list.append(list(i))
    return communities_list

communities = get_communities(G)

In [135]:
len(communities)

4

In [136]:
len(communities[0])

72577

In [137]:
len(communities[1])

29544

In [139]:
len(communities[2])

14284

In [140]:
len(communities[3])

4964